In [1]:
!apt-get update

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:3 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ Packages [43.6 kB]
Get:4 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Get:5 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:6 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:7 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Ign:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:9 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Get:10 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release [564 B]
Get:11 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release.gpg [833 B]
Hit:12 http://ppa.launchp

In [2]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://archive.apache.org/dist/spark/spark-2.4.6/spark-2.4.6-bin-hadoop2.7.tgz
!tar xf spark-2.4.6-bin-hadoop2.7.tgz

In [3]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.4.6-bin-hadoop2.7"

In [5]:
!pip install -q findspark
!pip install pyspark==2.4.7

     |████████████████████████████████| 217.9MB 66kB/s 
     |████████████████████████████████| 204kB 46.9MB/s 
  Created wheel for pyspark: filename=pyspark-2.4.7-py2.py3-none-any.whl size=218279465 sha256=4acaf6cf5969b093dc9dc742b1d10278e3a42028a3beff998c966f9b367a9723
  Stored in directory: /root/.cache/pip/wheels/34/1f/2e/1e7460f80acf26b08dbb8c53d7ff9e07146f2a68dd5c732be5
Successfully built pyspark


In [6]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


Change your directory to the Colab Data folder where you saved the CSV file.

In [11]:
os.chdir('/content/gdrive/MyDrive/')

In [9]:
# setting local session
APP_NAME = "Day83"
SPARK_URL = "local[*]"

In [12]:
# checking wd for videogames csv
os.getcwd()

'/content/gdrive/MyDrive'

In [7]:
from pyspark import SparkContext, SparkConf

In [13]:
# starting session
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName(APP_NAME).getOrCreate()

In [14]:
# load dataset
video = spark.read.format("csv").option("header", "true").option("inferSchema", "true").load("Video_Games_Sales_as_at_22_Dec_2016.csv")

In [15]:
# data review
video.show()

+--------------------+--------+---------------+------------+--------------------+--------+--------+--------+-----------+------------+------------+------------+----------+----------+-------------------+------+
|                Name|Platform|Year_of_Release|       Genre|           Publisher|NA_Sales|EU_Sales|JP_Sales|Other_Sales|Global_Sales|Critic_Score|Critic_Count|User_Score|User_Count|          Developer|Rating|
+--------------------+--------+---------------+------------+--------------------+--------+--------+--------+-----------+------------+------------+------------+----------+----------+-------------------+------+
|          Wii Sports|     Wii|           2006|      Sports|            Nintendo|   41.36|   28.96|    3.77|       8.45|       82.53|          76|          51|         8|       322|           Nintendo|     E|
|   Super Mario Bros.|     NES|           1985|    Platform|            Nintendo|   29.08|    3.58|    6.81|       0.77|       40.24|        null|        null|     

In [16]:
# schema
video.printSchema()

root
 |-- Name: string (nullable = true)
 |-- Platform: string (nullable = true)
 |-- Year_of_Release: string (nullable = true)
 |-- Genre: string (nullable = true)
 |-- Publisher: string (nullable = true)
 |-- NA_Sales: double (nullable = true)
 |-- EU_Sales: double (nullable = true)
 |-- JP_Sales: double (nullable = true)
 |-- Other_Sales: double (nullable = true)
 |-- Global_Sales: double (nullable = true)
 |-- Critic_Score: integer (nullable = true)
 |-- Critic_Count: integer (nullable = true)
 |-- User_Score: string (nullable = true)
 |-- User_Count: integer (nullable = true)
 |-- Developer: string (nullable = true)
 |-- Rating: string (nullable = true)



In [17]:
# printing first 10 rows
video.take(10)

[Row(Name='Wii Sports', Platform='Wii', Year_of_Release='2006', Genre='Sports', Publisher='Nintendo', NA_Sales=41.36, EU_Sales=28.96, JP_Sales=3.77, Other_Sales=8.45, Global_Sales=82.53, Critic_Score=76, Critic_Count=51, User_Score='8', User_Count=322, Developer='Nintendo', Rating='E'),
 Row(Name='Super Mario Bros.', Platform='NES', Year_of_Release='1985', Genre='Platform', Publisher='Nintendo', NA_Sales=29.08, EU_Sales=3.58, JP_Sales=6.81, Other_Sales=0.77, Global_Sales=40.24, Critic_Score=None, Critic_Count=None, User_Score=None, User_Count=None, Developer=None, Rating=None),
 Row(Name='Mario Kart Wii', Platform='Wii', Year_of_Release='2008', Genre='Racing', Publisher='Nintendo', NA_Sales=15.68, EU_Sales=12.76, JP_Sales=3.79, Other_Sales=3.29, Global_Sales=35.52, Critic_Score=82, Critic_Count=73, User_Score='8.3', User_Count=709, Developer='Nintendo', Rating='E'),
 Row(Name='Wii Sports Resort', Platform='Wii', Year_of_Release='2009', Genre='Sports', Publisher='Nintendo', NA_Sales=15.

In [19]:
video_1 = video.withColumnRenamed("Global_Sales", "Total_Sales")
video_1.show(10)

+--------------------+--------+---------------+------------+---------+--------+--------+--------+-----------+-----------+------------+------------+----------+----------+---------+------+
|                Name|Platform|Year_of_Release|       Genre|Publisher|NA_Sales|EU_Sales|JP_Sales|Other_Sales|Total_Sales|Critic_Score|Critic_Count|User_Score|User_Count|Developer|Rating|
+--------------------+--------+---------------+------------+---------+--------+--------+--------+-----------+-----------+------------+------------+----------+----------+---------+------+
|          Wii Sports|     Wii|           2006|      Sports| Nintendo|   41.36|   28.96|    3.77|       8.45|      82.53|          76|          51|         8|       322| Nintendo|     E|
|   Super Mario Bros.|     NES|           1985|    Platform| Nintendo|   29.08|    3.58|    6.81|       0.77|      40.24|        null|        null|      null|      null|     null|  null|
|      Mario Kart Wii|     Wii|           2008|      Racing| Nint

In [20]:
from pyspark.sql.functions import lit

video_1_lit = video_1.withColumn("Score_100", video_1["User_Score"] * lit(100))
video_1_lit.show(5)

+--------------------+--------+---------------+------------+---------+--------+--------+--------+-----------+-----------+------------+------------+----------+----------+---------+------+-----------------+
|                Name|Platform|Year_of_Release|       Genre|Publisher|NA_Sales|EU_Sales|JP_Sales|Other_Sales|Total_Sales|Critic_Score|Critic_Count|User_Score|User_Count|Developer|Rating|        Score_100|
+--------------------+--------+---------------+------------+---------+--------+--------+--------+-----------+-----------+------------+------------+----------+----------+---------+------+-----------------+
|          Wii Sports|     Wii|           2006|      Sports| Nintendo|   41.36|   28.96|    3.77|       8.45|      82.53|          76|          51|         8|       322| Nintendo|     E|            800.0|
|   Super Mario Bros.|     NES|           1985|    Platform| Nintendo|   29.08|    3.58|    6.81|       0.77|      40.24|        null|        null|      null|      null|     null| 

In [29]:
from pyspark.sql.functions import *

In [34]:
print((video_1_lit.count(), len(video_1_lit.columns)))

(16719, 17)


Print the percent of null values per column.

In [38]:
def null_value_perc(df):
    amount_missing = df.select([(count(when(isnan(c) | col(c).isNull(), c))/df.count()).alias(c) for c in df.columns])
    amount_missing.show()

null_value_perc(video_1_lit)

+--------------------+--------+---------------+--------------------+---------+--------+--------+--------+-----------+-----------+------------------+------------------+-------------------+------------------+------------------+-------------------+------------------+
|                Name|Platform|Year_of_Release|               Genre|Publisher|NA_Sales|EU_Sales|JP_Sales|Other_Sales|Total_Sales|      Critic_Score|      Critic_Count|         User_Score|        User_Count|         Developer|             Rating|         Score_100|
+--------------------+--------+---------------+--------------------+---------+--------+--------+--------+-----------+-----------+------------------+------------------+-------------------+------------------+------------------+-------------------+------------------+
|1.196243794485316...|     0.0|            0.0|1.196243794485316...|      0.0|     0.0|     0.0|     0.0|        0.0|        0.0|0.5133082122136492|0.5133082122136492|0.40098091991147794|0.5460254799928226

In [44]:
video_1_lit_ = video_1_lit.na.drop(subset=['Developer'])

In [45]:
null_value_perc(video_1_lit_)

+----+--------+---------------+-----+---------+--------+--------+--------+-----------+-----------+-------------------+-------------------+--------------------+------------------+---------+--------------------+------------------+
|Name|Platform|Year_of_Release|Genre|Publisher|NA_Sales|EU_Sales|JP_Sales|Other_Sales|Total_Sales|       Critic_Score|       Critic_Count|          User_Score|        User_Count|Developer|              Rating|         Score_100|
+----+--------+---------------+-----+---------+--------+--------+--------+-----------+-----------+-------------------+-------------------+--------------------+------------------+---------+--------------------+------------------+
| 0.0|     0.0|            0.0|  0.0|      0.0|     0.0|     0.0|     0.0|        0.0|        0.0|0.19463153724247226|0.19463153724247226|0.011291600633914422|0.2493066561014263|      0.0|0.014461172741679873|0.2493066561014263|
+----+--------+---------------+-----+---------+--------+--------+--------+----------

Calculate the mean global sales by developer using a groupby aggregation and show the results.

In [46]:
video_1_lit_.groupBy("Developer").mean("Total_Sales").show()

+--------------------+-------------------+
|           Developer|   avg(Total_Sales)|
+--------------------+-------------------+
| Hyperspace Cowgirls|                0.3|
|            Deep Red|               0.02|
|        Funbox Media|               0.02|
|      Camouflaj, LLC|               0.01|
|          SCEA, Sims|                0.7|
|Ubisoft Barcelona...|               0.47|
|LightBox Interactive|               0.29|
|      Handheld Games|0.12333333333333335|
|     Beep Industries|               0.12|
|          SpiderMonk|               0.07|
|             Blue 52|               0.04|
|      Rockstar Leeds|  2.622222222222222|
|           Hangar 13| 0.8733333333333334|
|         id Software|               0.56|
|       ZeniMax Media|               1.09|
|Gearbox Software,...| 0.6666666666666666|
|           Telegames|               0.29|
|                 3DO|0.16470588235294115|
|            Tantatus|0.17750000000000002|
|     Lancarse, Atlus|               0.33|
+----------

In [47]:
video_1_lit_.show(5)

+--------------------+--------+---------------+--------+---------+--------+--------+--------+-----------+-----------+------------+------------+----------+----------+---------+------+-----------------+
|                Name|Platform|Year_of_Release|   Genre|Publisher|NA_Sales|EU_Sales|JP_Sales|Other_Sales|Total_Sales|Critic_Score|Critic_Count|User_Score|User_Count|Developer|Rating|        Score_100|
+--------------------+--------+---------------+--------+---------+--------+--------+--------+-----------+-----------+------------+------------+----------+----------+---------+------+-----------------+
|          Wii Sports|     Wii|           2006|  Sports| Nintendo|   41.36|   28.96|    3.77|       8.45|      82.53|          76|          51|         8|       322| Nintendo|     E|            800.0|
|      Mario Kart Wii|     Wii|           2008|  Racing| Nintendo|   15.68|   12.76|    3.79|       3.29|      35.52|          82|          73|       8.3|       709| Nintendo|     E|830.0000000000